In [1]:
import pandas as pd
import numpy as np
from scipy import signal, misc
from datetime import datetime, timedelta

import os
import json

In [2]:
prom_data=pd.read_csv("Device_200011546604332_2019_09_21.csv")
pdf=pd.DataFrame({
    'timestamp':prom_data.TableTimeStamp,
    'milk_temp':prom_data.Parameter2,
    'tank_switch':prom_data.Parameter11,
    'agitator':prom_data.Parameter12,
    'discharge_pump':prom_data.Parameter15,
    'compressor':prom_data.Parameter6,
    'ac_voltage':prom_data.Parameter5,
    'pump_current':prom_data.Parameter7,
    'tss_temp':prom_data.Parameter1
})
ascendo_data=pd.read_csv("Ascendo_200011546604332_2019_09_21.csv")

# Creating events

In [3]:
import parameters

def milk_edgecase(milk_temp_mean):

    if int(milk_temp_mean) in range(parameters.safe_milk_temp_low,parameters.safe_milk_temp_high): #safe temperature
        return 0
    elif milk_temp_mean >parameters.safe_milk_temp_high: #OVER upperbound - hotter product
        return 1
    else: #BELOW lowerbound - cold product
        return 2
    
def milk_present(milk_temp_mean):
    if int(milk_temp_mean) in range(parameters.milk_not_present_low,parameters.milk_not_present_high): #no product!!!!! Made changes here
        return 0
    elif milk_temp_mean in range(parameters.fresh_product_low, parameters.fresh_product_high): #fresh product poured in
        return 2
    else:
        return 1
    
def create_life_events(new_df):
    
    new_df['milk_present']=new_df['milk_temp_mean'].apply(milk_present)
    new_df['milk_temp_edgecase']=new_df['milk_temp_mean'].apply(milk_edgecase)
    new_val=[]
    
    i=0
    for row in new_df.milk_present:
        #milk_present=3 indicating milk lift
        if (new_df.milk_present.iloc[i-1]==1)&(row==0):
            new_val.append(3)
        #setting milk_present=1 for when it gets 10-25 range after fresh milk is poured
        elif (new_df.milk_present.iloc[i-1]==2)&(row==0):
            new_val.append(1)
        else:
            new_val.append(row)
        i=i+1
        
    new_df['milk_present']=new_val      
    cooling_process=[]
    #tank switch on and discharge pump on
    for row in new_df.iterrows():

        if (row[1].tank_switch_mean!=0.0) &(row[1].discharge_pump_mean!=0.0):
            cooling_process.append(1) #cooling process in process
        else:
            cooling_process.append(0) 
    new_df['cooling']=cooling_process

    #creatn empty list to fill values into device_life_events table
    datetimelist=[]
    event_id=[]

    
    return new_df
   
life_events= create_life_events(ascendo_data)  


In [4]:
life_events[['milk_temp_mean','tank_switch_mean','discharge_pump_mean','milk_present','milk_temp_edgecase','cooling']]

,milk_temp_mean,tank_switch_mean,discharge_pump_mean,milk_present,milk_temp_edgecase,cooling
0,6,1,0,1,0,0
1,4,1,0,1,0,0
2,5,1,0,1,0,0
3,5,1,0,1,0,0
4,5,1,0,1,0,0
5,4,1,0,1,0,0
6,5,1,0,1,0,0
7,5,1,0,1,0,0
8,6,1,1,1,0,1
9,9,1,1,1,1,1


In [5]:
incident_details=pd.read_csv("incident_details.csv")
incident_details.drop(labels=['promethean_device_id','end_customer_id','created_by','created_on','updated_by','updated_on'], axis=1, inplace=True)
incident_details

,id,incident_name,incident_type,description,priority,threshold_time,threshold_count,threshold_time_units
0,1,AC Voltage if turned OFF Often,"Milk Temperature over 6,but no AC",Site,High,9,7,Days
1,2,Pattern of Discharge pump going OFF,"Milk Temperature over 6,but discharge pump OFF",System,High,1,1,Days
2,3,Pattern of Tank Switch being turned OFF,Milk present but Tank Switch OFF,System,Critical,3,7,Days
3,4,Pattern of not enough cooling,Milk present but cooling hasnt began,System,Critical,1,1,Days
4,5,Pattern of Tank switch being ON when not required,No Milk but Tank Switch is ON,System,High,2,7,Days
5,6,Tank Switch ON more than 3 hours,No Milk but Tank Switch is ON for more than 3 ...,System,Critical,1,1,Days
6,7,Cyclic mode is going OFF when it should not be,Milk temperature in safe range but Agitator Cy...,System,Critical,1,1,Days
7,8,Agitator ON for too long,Avgerage time for Agitator to get into ACM is ...,System,Pattern,1,1,Days
8,9,Charging TSS is taking longer than normal,Avgerage time for TSS to charge is long,System,Pattern,1,1,Days
9,10,Tank switch ON during milk lift,Tank switch and agitator ON during milk lift,System,Critical,3,7,Days


# Creating incidents

In [6]:
def incident_6(new_df):
    incident_6=[]
    count_6=0
    i=1
    for t in new_df.iterrows():
        if (t[1].milk_present ==0)  & (t[1].tank_switch_mean==1) :  #incident 6
            incident_6.append(1)
        else:
            incident_6.append(0)
        i=i+1

    new_val=[]
    i=1

    for t in new_df.iterrows():
        if (t[1].milk_present ==0)  & (t[1].tank_switch_mean==1) :  #incident 6    
            count_6=count_6+1
            if (count_6>parameters.tank_switch_on_for_max_hours)&(incident_6[i-1]==1): #checks for tank_switch_on_for_max_hours hours
                new_val.append(1)
            else:
                new_val.append(0)
        else:
                new_val.append(0) 
                count_6=0
        i=i+1  
    new_df['incident_6']=new_val 
    return new_df

In [7]:
def create_incidents(new_df):
    incident_log=pd.DataFrame(columns=['timestamp','incident_id','incident_name'])
    new_df['incident_6']=incident_6(new_df)

    i=0
    for t in new_df.iterrows():
    
        # milk_present==1  AND milk_temp_edgecase ==0 AND AC_voltage not in range
        if (t[1].milk_present ==1) & (t[1].milk_temp_edgecase==1) & (int(t[1].ac_voltage_mean) not in range(parameters.ac_voltage_low, parameters.ac_voltage_high)):  #incident 1
                    incident_log.loc[i]=(t[1].timestamp,1,'AC Voltage if turned OFF Often')
                    i=i+1
        
        if (t[1].milk_present ==1) & (t[1].milk_temp_mean>parameters.safe_milk_temp_high) & (t[1].discharge_pump_mean==0):  #incident 2
                    incident_log.loc[i]=(t[1].timestamp,2,'Pattern of Discharge pump going OFF')
                    i=i+1

        if (t[1].milk_present ==1)  & (t[1].tank_switch_mean==0):  #incident 3
                    incident_log.loc[i]=(t[1].timestamp,3,'Pattern of Tank Switch being turned OFF')
                    i=i+1
                
        if (t[1].milk_present ==1)  & (t[1].tank_switch_mean==0) & (t[1].discharge_pump_mean==0):  #incident 4
                    incident_log.loc[i]=(t[1].timestamp,4,'Pattern of not enough cooling')
                    i=i+1
                
        if (t[1].milk_present ==0)  & (t[1].tank_switch_mean==1) :  #incident 5
                    incident_log.loc[i]=(t[1].timestamp,5,'Pattern of Tank switch being ON when not required')
                    i=i+1
                
        if (t[1].incident_6==1): #incident 6
                    incident_log.loc[i]=(t[1].timestamp,6,'Tank Switch ON more than 3 hours')
                    i=i+1
            
        if (t[1].milk_present==3)&(t[1].tank_switch_mean==1):  #incident 10
                    incident_log.loc[i]=(t[1].timestamp,10,'Tank switch ON during milk lift')
                    i=i+1
                
    return incident_log

incident_log=create_incidents(life_events)
incident_log

,timestamp,incident_id,incident_name
0,2019-09-21 14:00:00,10,Tank switch ON during milk lift
